# Trampoline no name match

### Imports

In [531]:
import pandas as pd
import plotly.express as px
import os
import sqlite3
import hashlib
import matplotlib.pyplot as plt
import numpy as np
from decimal import Decimal

### Loading DATs

In [634]:
# dataFolder = "/Users/falkowork/Downloads/WCH Tokyo TRA Herren/Day 1 TRA Men/"
dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
db_name = "2020_olympic_games_men"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
connection = sqlite3.connect("trampoline.db")
cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='Olympic Games') & (df['Location']=='Tokyo') & (df['Gender']=='Men')]

In [635]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        if len(df_trampoline.columns) > 2:
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st'
                    phase = 'Qualification'
                elif len(check_df) == 1:
                    routine = '2nd'
                    phase = 'Qualification'
                elif len(check_df) == 2:
                    routine = '1st'
                    phase = 'Final'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1


df_main.to_sql(name=db_name, con=connection)
connection.close()
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))

H: 4200, T: 7980
H: 2300, T: 3435
H: 4800, T: 8600
H: 5700, T: 10155
H: 3700, T: 6490
H: 4600, T: 8500
H: 2700, T: 5350
H: 2900, T: 5890
H: 7200, T: 14135
H: 9600, T: 18580
H: 9000, T: 17895
H: 9500, T: 15810
H: 9400, T: 18070
H: 9900, T: 17550
H: 9700, T: 18130
H: 4800, T: 8580
H: 4500, T: 8335
H: 3300, T: 6475
H: 3500, T: 10835
H: 9600, T: 16965
H: 7400, T: 12765
H: 3000, T: 4790
H: 2700, T: 4540
H: 9800, T: 17815
H: 9200, T: 19115
H: 9700, T: 17805
H: 9500, T: 18435
H: 9000, T: 18065
H: 9700, T: 17785
H: 9800, T: 17875
H: 9600, T: 18745
H: 1000, T: 1890
H: 3400, T: 6915
H: 9400, T: 17420
H: 9300, T: 16690
H: 9000, T: 14980
H: 9000, T: 16675
H: 8600, T: 15780
H: 9100, T: 17350
H: 9500, T: 16805
H: 8900, T: 17640
H: 9400, T: 17335
H: 6500, T: 12285
H: 4600, T: 8885
H: 6400, T: 12085
H: 9200, T: 16675
H: 1500, T: 3575
H: 1800, T: 3410
H: 1600, T: 2795
H: 1800, T: 3375
H: 3500, T: 5940
H: 2800, T: 4600
H: 2700, T: 4380
H: 900, T: 1365
H: 8900, T: 17100
H: 8500, T: 16735
H: 8700, T: 1691

In [115]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        if len(df_trampoline.columns) > 2:
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(df_wctokyo['H']*1000==sum_h) & (df_wctokyo['T']*1000==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                # df_trampoline.to_sql(name=hash_val, con=connection)*

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st'
                    phase = 'Qualification'
                elif len(check_df) == 1:
                    routine = '2nd'
                    phase = 'Qualification'
                elif len(check_df) == 2:
                    routine = '1st'
                    phase = 'Final'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1


# df_main.to_sql(name=db_name, con=connection)
connection.close()
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))

H: 9200, T: 16190
H: 9300, T: 18045
H: 8700, T: 17405
H: 8600, T: 17970
H: 9600, T: 16835
H: 9300, T: 17580
H: 9300, T: 17240
H: 9000, T: 17220
H: 9400, T: 18205
H: 9100, T: 17730
H: 8900, T: 16295
H: 8800, T: 18640
H: 9100, T: 17605
H: 9100, T: 15460
H: 2000, T: 3595
H: 8900, T: 16480
H: 9700, T: 16745
H: 7400, T: 12520
H: 9100, T: 16025
H: 5400, T: 9590
H: 9300, T: 16120
H: 9500, T: 16670
H: 8900, T: 16700
H: 9000, T: 15660
H: 9500, T: 16875
H: 8800, T: 15985
H: 4900, T: 11610
H: 9200, T: 17150
H: 9400, T: 15590
H: 9200, T: 18290
H: 9600, T: 17485
H: 9800, T: 17605
H: 9600, T: 18025
H: 9700, T: 18690
H: 9700, T: 18270
H: 9500, T: 15935
H: 1000, T: 1695
H: 9300, T: 16545
H: 9800, T: 16515
H: 9700, T: 16000
H: 9700, T: 17185
H: 9900, T: 17705
H: 9000, T: 17220
H: 5400, T: 16515
H: 9500, T: 17635
H: 9200, T: 16990
H: 9000, T: 15940
H: 8900, T: 17010
H: 9500, T: 18065
H: 9300, T: 17165
H: 9600, T: 16530
H: 9300, T: 18180
H: 9800, T: 16245
H: 6600, T: 12230
H: 9400, T: 17375
H: 9400, T: 1

# Women Olympia

In [636]:
# dataFolder = "/Users/falkowork/Downloads/WCH Tokyo TRA Herren/Day 1 TRA Men/"
dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Women_EuroTrampMachines_Logs/"
db_name = "2020_olympic_games_women"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
connection = sqlite3.connect("trampoline.db")
cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='Olympic Games') & (df['Location']=='Tokyo') & (df['Gender']=='Women')]


In [637]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        if len(df_trampoline.columns) > 2:
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]
            if dataset == '20210730_135416.dat':
                print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st'
                    phase = 'Qualification'
                elif len(check_df) == 1:
                    routine = '2nd'
                    phase = 'Qualification'
                elif len(check_df) == 2:
                    routine = '1st'
                    phase = 'Final'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1


df_main.to_sql(name=db_name, con=connection)
connection.close()
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))

H: 9400, T: 16205
Multi Match: 0, No Match: 22, All: 74, Matched: 39


# WCH Tokyo Men

In [638]:
dataFolder = "/Users/falkowork/Downloads/Data/WCH Tokyo TRA Herren/Day 1 TRA Men/"
# dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
db_name = "2019_world_championships_men"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
connection = sqlite3.connect("trampoline.db")
cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='World Championships') & (df['Location']=='Tokyo') & (df['Gender']=='Men')]
df_overall = pd.DataFrame(columns=header)


In [639]:
def stripname(name):
    # print(name)
    name = name.strip().split()
    return name[0].capitalize() + " " + name[1].capitalize()
    # print(name)

startlist = pd.read_csv("WCH_Tokyo_Startlist_Men.csv", header=None)
header = ('Start', 'Start Number', 'Name', 'Country')
startlist.columns = header
# startlist["Name"] = startlist.apply(lambda x: x["Name"].strip(), axis=1)
startlist["Name"] = startlist.apply(lambda x: stripname(x["Name"]), axis=1)
group_list = []
group = 0
last_idx = 0
routine = '1st'
routine_list = []
groups = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]
for entry in startlist["Start"]:
    
    # print("Last Index {0} - Entry {1}".format(last_idx, entry))
    if last_idx >= entry:
        last_idx = 0
        group += 1
        if routine == '1st':
            routine = '2nd'
        elif routine == '2nd':
            routine = '1st'
    else:
        last_idx = entry

    group_list.append(groups[group])
    routine_list.append(routine)

startlist["Group"] = group_list
startlist["Routine"] = routine_list

In [640]:
df_dat = pd.DataFrame(columns=('Filename', 'T', 'H'))
filenames = []
t_list = []
h_list = []
hash_table = []
for dataset in sortedList:
        if not ".DS_Store" in dataset:
            datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
            trampoline_data = datContent[4:14]
            df_trampoline = pd.DataFrame(trampoline_data)
            if len(df_trampoline.columns) > 2:
                df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
                header = ("T", "H", "x", "y")
                df_trampoline.columns = header
                sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
                sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
                t_list.append(sum_t)
                h_list.append(sum_h)
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                hash_table.append(hash_val)
                df_trampoline.to_sql(name=hash_val, con=connection)
                
            else:
                t_list.append(0)
                h_list.append(0)
                hash_table.append('')
            filenames.append(dataset)
df_dat['Filename'] = filenames
df_dat['H'] = h_list
df_dat['T'] = t_list
df_dat['Hash'] = hash_table

In [641]:
df_datsave = df_dat.copy()
h_rating = []
t_rating = []
dataset_list = []
matched = 0
hash_list = []
df_wctokyo["Name"] = df_wctokyo.apply(lambda x: stripname(x["Name"]), axis=1)
for idx, entry in startlist.iterrows():
    
    entry_with_rating = df_wctokyo[(df_wctokyo['Name']==entry['Name']) & (df_wctokyo['Routine']==entry['Routine'])]
    
    if len(entry_with_rating) > 0:
        h_rating.append(entry_with_rating['H'].iloc[0])
        t_rating.append(entry_with_rating['T'].iloc[0])

        h_cents = int(round(entry_with_rating['H'].iloc[0]*1000))
        t_cents = int(round(entry_with_rating['T'].iloc[0]*1000))

        df_select = df_datsave[((df_datsave['H']==h_cents) & (df_datsave['T']==t_cents))]
        
        if len(df_select) > 0:
            dataset_list.append(df_select['Filename'].iloc[0])
            hash_list.append(df_select['Hash'].iloc[0])
            # df_datsave = df_datsave.drop([df_select.index[0]])
            matched += 1
        elif len(df_select) == 0:
            dataset_list.append('')
            hash_list.append('')
    elif len(entry_with_rating) == 0:
        h_rating.append(0)
        t_rating.append(0)
        dataset_list.append('')
        hash_list.append('')
    


            

startlist['H'] = h_rating
startlist['T'] = t_rating
startlist['Filename'] = dataset_list
startlist["Phase"] = "Qualification"
startlist['Hash'] = hash_list
startlist = startlist.drop(["Start", "Filename", "Group", 'T', 'H'], axis=1)
# startlist.to_sql(name=db_name, con=connection)
# connection.close()
# print(matched)
df_overall = pd.concat([df_overall, startlist])


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


# Semi


In [642]:
dataFolder = "/Users/falkowork/Downloads/Data/WCH Tokyo TRA Herren/Day 3 TRA Men/"
# dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
# db_name = "2019_world_championships_men"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
# connection = sqlite3.connect("trampoline.db")
# cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='World Championships') & (df['Location']=='Tokyo') & (df['Gender']=='Men') & (df['Phase']=='Semi')]

In [643]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        which_competition = datContent[0][-1]
        if (len(df_trampoline.columns) > 2 and int(which_competition) < 3):
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st'
                    phase = 'Qualification'
                elif len(check_df) == 1:
                    routine = '2nd'
                    phase = 'Qualification'
                elif len(check_df) == 2:
                    routine = '1st'
                    phase = 'Final'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1

df_main["Phase"] = 'Semi'
# df_main.to_sql(name=db_name, con=connection)
df_overall = pd.concat([df_overall, df_main])
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))

H: 8900, T: 16550
H: 1000, T: 1840
H: 8900, T: 15800
H: 9400, T: 17290
H: 9000, T: 17225
H: 9100, T: 15920
H: 9400, T: 16280
H: 9200, T: 17130
H: 9100, T: 16745
H: 9400, T: 16475
H: 9000, T: 17155
H: 9400, T: 17300
H: 1000, T: 1870
H: 9400, T: 17080
H: 9500, T: 16795
H: 9100, T: 17415
H: 9200, T: 16755
H: 9300, T: 16870
H: 9200, T: 16735
H: 9400, T: 16845
H: 9100, T: 17700
H: 9800, T: 17090
H: 9700, T: 17760
H: 8700, T: 16755
Multi Match: 0, No Match: 0, All: 32, Matched: 24


# Final

In [644]:
dataFolder = "/Users/falkowork/Downloads/Data/WCH Tokyo TRA Herren/Day 4 TRA Men/"
# dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
# db_name = "2019_world_championships_women"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
# connection = sqlite3.connect("trampoline.db")
# cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='World Championships') & (df['Location']=='Tokyo') & (df['Gender']=='Men') & (df['Phase']=='Final')]

In [645]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        which_competition = datContent[0][-1]
        if (len(df_trampoline.columns) > 2 and int(which_competition) < 3):
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st' 
                elif len(check_df) == 1:
                    routine = '2nd'
                elif len(check_df) == 2:
                    routine = '1st'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1


df_main["Phase"] = 'Final'


df_overall = pd.concat([df_overall, df_main])
df_overall["Gender"] = 'Men'
df_overall.to_sql(name=db_name, con=connection)
connection.close()
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))


H: 9200, T: 17330
H: 9300, T: 17185
H: 6500, T: 12135
H: 8900, T: 16750
H: 9700, T: 17050
H: 9500, T: 16565
H: 9500, T: 17920
H: 9600, T: 17905
H: 2700, T: 5325
H: 9900, T: 17795
H: 9100, T: 16415
H: 1600, T: 3465
H: 9000, T: 17125
Multi Match: 0, No Match: 5, All: 18, Matched: 8


# WCH Tokyo Women Qualification

In [646]:
dataFolder = "/Users/falkowork/Downloads/Data/WCH Tokyo TRA Damen/Day 1 TRA Women/"
# dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
db_name = "2019_world_championships_women"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
connection = sqlite3.connect("trampoline.db")
cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='World Championships') & (df['Location']=='Tokyo') & (df['Gender']=='Women')]
df_overall = pd.DataFrame(columns=header)


In [647]:
# def stripname(name):
#     # print(name)
#     name = name.strip().split()
#     return name[0].capitalize() + " " + name[1].capitalize()
#     # print(name)

startlist = pd.read_csv("WCH_Tokyo_Startlist_Women.csv", header=None)
header = ('Start', 'Start Number', 'Name', 'Country')
startlist.columns = header
# startlist["Name"] = startlist.apply(lambda x: x["Name"].strip(), axis=1)
startlist["Name"] = startlist.apply(lambda x: stripname(x["Name"]), axis=1)
group_list = []
group = 0
last_idx = 0
routine = '1st'
routine_list = []
groups = [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9]
for entry in startlist["Start"]:
    
    # print("Last Index {0} - Entry {1}".format(last_idx, entry))
    if last_idx >= entry:
        last_idx = 0
        group += 1
        if routine == '1st':
            routine = '2nd'
        elif routine == '2nd':
            routine = '1st'
    else:
        last_idx = entry

    group_list.append(groups[group])
    routine_list.append(routine)

startlist["Group"] = group_list
startlist["Routine"] = routine_list

In [648]:
df_dat = pd.DataFrame(columns=('Filename', 'T', 'H'))
filenames = []
t_list = []
h_list = []
hash_table = []
for dataset in sortedList:
        if not ".DS_Store" in dataset:
            datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
            trampoline_data = datContent[4:14]
            df_trampoline = pd.DataFrame(trampoline_data)
            if len(df_trampoline.columns) > 2:
                df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
                header = ("T", "H", "x", "y")
                df_trampoline.columns = header
                sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
                sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
                t_list.append(sum_t)
                h_list.append(sum_h)
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                hash_table.append(hash_val)
                df_trampoline.to_sql(name=hash_val, con=connection)
                
            else:
                t_list.append(0)
                h_list.append(0)
                hash_table.append('')
            filenames.append(dataset)
df_dat['Filename'] = filenames
df_dat['H'] = h_list
df_dat['T'] = t_list
df_dat['Hash'] = hash_table

In [649]:
df_datsave = df_dat.copy()
h_rating = []
t_rating = []
dataset_list = []
matched = 0
hash_list = []
df_wctokyo["Name"] = df_wctokyo.apply(lambda x: stripname(x["Name"]), axis=1)
for idx, entry in startlist.iterrows():
    # print(entry["Name"])
    # print(entry["Routine"])
    entry_with_rating = df_wctokyo[(df_wctokyo['Name']==entry['Name']) & (df_wctokyo['Routine']==entry['Routine'])]
    # print("Startlist  Name: " + entry['Name'])
    if len(entry_with_rating) > 0:
        # print("Ratinglist Name: "+ entry_with_rating['Name'].iloc[0])
        h_rating.append(entry_with_rating['H'].iloc[0])
        t_rating.append(entry_with_rating['T'].iloc[0])
        # test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]

        h_cents = int(round(entry_with_rating['H'].iloc[0]*1000))
        t_cents = int(round(entry_with_rating['T'].iloc[0]*1000))

        df_select = df_datsave[((df_datsave['H']==h_cents) & (df_datsave['T']==t_cents))]
        # print(df_select)
        # print('H: {0}, T: {1}'.format(h_cents, t_cents))
        if len(df_select) > 0:
            # print('Match found')
            dataset_list.append(df_select['Filename'].iloc[0])
            # print(df_select['Hash'])
            hash_list.append(df_select['Hash'].iloc[0])
            df_datsave = df_datsave.drop([df_select.index[0]])
            matched += 1
        elif len(df_select) == 0:
            # print('H: {0} vs {1}, T: {2} vs {3}'.format(df_datsave['H'], h_cents, df_datsave['T'], t_cents))
            dataset_list.append('')
            hash_list.append('')
    elif len(entry_with_rating) == 0:
        # print('Name not found ' + entry['Name'] + 'at Routine: ' + entry['Routine'])
        h_rating.append(0)
        t_rating.append(0)
        dataset_list.append('')
        hash_list.append('')
    


            

startlist['H'] = h_rating
startlist['T'] = t_rating
startlist['Filename'] = dataset_list
startlist["Phase"] = "Qualification"
startlist['Hash'] = hash_list
startlist = startlist.drop(["Start", "Filename", "Group", 'T', 'H'], axis=1)
# startlist.to_sql(name=db_name, con=connection)
# connection.close()
# print(matched)
df_overall = pd.concat([df_overall, startlist])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [650]:
#

# Semi

In [651]:
dataFolder = "/Users/falkowork/Downloads/Data/WCH Tokyo TRA Damen/Day 3 TRA Women/"
# dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
db_name = "2019_world_championships_women"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
# connection = sqlite3.connect("trampoline.db")
# cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='World Championships') & (df['Location']=='Tokyo') & (df['Gender']=='Women') & (df['Phase']=='Semi')]

In [652]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        which_competition = datContent[0][-1]
        if (len(df_trampoline.columns) > 2 and int(which_competition) < 3):
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st'
                    phase = 'Qualification'
                elif len(check_df) == 1:
                    routine = '2nd'
                    phase = 'Qualification'
                elif len(check_df) == 2:
                    routine = '1st'
                    phase = 'Final'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1

df_main["Phase"] = 'Semi'
# df_main.to_sql(name=db_name, con=connection)
# connection.close()
df_overall = pd.concat([df_overall, df_main])
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))

H: 9000, T: 15130
H: 9200, T: 15095
H: 1600, T: 3285
H: 4300, T: 7580
H: 9300, T: 15675
H: 4500, T: 7985
H: 9200, T: 14775
H: 9400, T: 15515
H: 9500, T: 15190
H: 5800, T: 9245
H: 9300, T: 15330
H: 9100, T: 14750
H: 9500, T: 15340
H: 8900, T: 15325
H: 9200, T: 15380
H: 9500, T: 15230
H: 9100, T: 15165
H: 9300, T: 15665
H: 9600, T: 14880
H: 9300, T: 15390
H: 8200, T: 14330
H: 9600, T: 15990
H: 9600, T: 16120
H: 2800, T: 5065
Multi Match: 0, No Match: 1, All: 33, Matched: 23


# Final

In [653]:
dataFolder = "/Users/falkowork/Downloads/Data/WCH Tokyo TRA Damen/Day 4 TRA Women/"
# dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
db_name = "2019_world_championships_women"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
# connection = sqlite3.connect("trampoline.db")
# cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='World Championships') & (df['Location']=='Tokyo') & (df['Gender']=='Women') & (df['Phase']=='Final')]

In [654]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        which_competition = datContent[0][-1]
        if (len(df_trampoline.columns) > 2 and int(which_competition) < 3):
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(round(df_wctokyo['H']*1000).astype(int)==sum_h) & (round(df_wctokyo['T']*1000).astype(int)==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st' 
                elif len(check_df) == 1:
                    routine = '2nd'
                elif len(check_df) == 2:
                    routine = '1st'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1


df_main["Phase"] = 'Final'

# connection.close()
df_overall = pd.concat([df_overall, df_main])
df_overall["Gender"] = 'Women'
df_overall.to_sql(name=db_name, con=connection)
connection.close()
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))


H: 8900, T: 14850
H: 1000, T: 1685
H: 9600, T: 15525
H: 8800, T: 15320
H: 9500, T: 15220
H: 9200, T: 15290
H: 9500, T: 15860
H: 8900, T: 15905
H: 9300, T: 14860
H: 9500, T: 15120
H: 9000, T: 15965
H: 9600, T: 15200
H: 9200, T: 15530
Multi Match: 0, No Match: 5, All: 17, Matched: 8


In [580]:
df_overall

,Start Number,Gender,Country,Phase,Routine,Name,Hash
0,334,NaN,UZB,Qualification,1st,Masharipov Jurabek,
1,246,NaN,GER,Qualification,1st,Fritzsche Lars,
2,221,NaN,FRA,Qualification,1st,Gouzou Pierre,
3,327,NaN,USA,Qualification,1st,Shostak Aliaksei,
4,337,NaN,UZB,Qualification,1st,Usmonov Samirbek,
...,...,...,...,...,...,...,...
19,0,Men,CHN,Semi,1st,Tu Xiao,c9a5498999154ca66a12e30df8b883df
20,0,Men,BLR,Semi,1st,Litvinovich Ivan,be48c36060baa43cb1dc10fb1810cc77
21,0,Men,CHN,Semi,1st,Dong Dong,3cf573c25625df9b16a7fa6a83c8c85f
22,0,Men,CHN,Semi,1st,Gao Lei,a467744205cb373887262566bf7462ba


In [579]:
df_overall = pd.concat([df_overall, df_main])

In [583]:
df_overall

,Start Number,Gender,Country,Phase,Routine,Name,Hash
0,334,NaN,UZB,Qualification,1st,Masharipov Jurabek,
1,246,NaN,GER,Qualification,1st,Fritzsche Lars,
2,221,NaN,FRA,Qualification,1st,Gouzou Pierre,
3,327,NaN,USA,Qualification,1st,Shostak Aliaksei,
4,337,NaN,UZB,Qualification,1st,Usmonov Samirbek,
...,...,...,...,...,...,...,...
3,0,Men,RUS,Final,1st,Ushakov Dmitrii,811799b1f61ad340e350eba7ff7ff1ea
4,0,Men,CHN,Final,1st,Dong Dong,d7af8651f9f8a27968421f562b63a5d4
5,0,Men,RUS,Final,1st,Yudin Andrey,f595fb075ccc9a8a64a12de29f82812b
6,0,Men,BLR,Final,1st,Litvinovich Ivan,18d88c054001712303839c42e498f802
